In [1]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob


In [2]:
WEAK = False

x_len = 5000
t_len = 6000

xs = np.arange(x_len)
dt = 1

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]  

if WEAK:
    X, T = np.meshgrid(xs, np.arange(t_len))
    XT = np.asarray([X, T]).T

    pde_lib = ps.WeakPDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatiotemporal_grid=XT,
        is_uniform=True,
        K=1000,
    )
else:
    
    pde_lib = ps.PDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatial_grid=xs,
        include_bias=True,
        is_uniform=True,
    )
    

In [3]:
def printEnsemble(model, cut_off = 1e-3, median= False):
    if median:
        coefs = np.median(model.coef_list, axis=0)[0,:]
    else:
        coefs = np.mean(model.coef_list, axis=0)[0,:]
    above_cut_off = np.argwhere(np.abs(coefs)>cut_off).flatten()
    features =  model.get_feature_names()
    equation_str= "(x0)' = "
    for count, val in enumerate(above_cut_off):
        if count >0:
            equation_str+=" + "
        equation_str+= "{0:.4f}".format(coefs[val])
        equation_str+= " "+features[val]
        
    print(equation_str)

In [4]:
def saveEnsemble(model,fileStr,thresh):
    np.save("/home/ldoyle/notebooks/ensembleSINDyArrays_50/"+fileStr+"_thresh_"+"{:.2E}".format(thresh), model.coef_list)
#     print(fileStr+"_thresh_"+"{:.2E}".format(thresh))

In [5]:
def SINDyImplementEnsemble(dataset, fileStr,thresh, alph=0.001, n_models=30, time_steps=400, stlsq_max =100,median=True):
    optimizer = ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max)
#     optimizer = ps.EnsembleOptimizer( opt=ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max),bagging=True,  n_models = n_models,n_subset =time_steps)  
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, 
                     differentiation_method= ps.differentiation.SmoothedFiniteDifference())
    model.fit(dataset, t=dt,ensemble=True, n_models=n_models, n_subset=time_steps)
    print("Ensemble STLSQ with Threshold "+str(thresh))
    printEnsemble(model,median=median)
    saveEnsemble(model,fileStr,thresh)

## Unfiltered

### Dataset 1

In [6]:
path = "/data/data2/south-data-ejm/hdd/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/"

fileStr = "2021-11-01T2311_unfiltered"

init = 11
final= 16

time_subsample = 10
full_dat = np.zeros((5000,(12000//time_subsample)*(final-init)))

k = 0
for i in np.arange(init,final):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    
working_dat = full_dat/np.std(full_dat)


In [7]:
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [8]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6433 x0_1 + 0.3577 x0_111


In [9]:
thresh = 1.e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6425 x0_1 + 0.3456 x0_111


In [10]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6238 x0_1 + 0.3505 x0_111 + 0.0086 x0x0_1 + 0.0101 x0x0x0_1 + -0.0039 x0x0_111 + 0.0031 x0x0x0_111


In [11]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6439 x0_1 + -0.0073 x0_11 + 0.3587 x0_111 + -0.0024 x0_1111 + 0.0160 x0x0_1 + -0.0042 x0x0_11 + 0.0020 x0x0_111


In [12]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.0014 1 + -0.0029 x0 + 0.6400 x0_1 + 0.0075 x0_11 + 0.3563 x0_111 + 0.0016 x0_1111 + 0.0035 x0x0_1 + 0.0108 x0x0x0_1 + -0.0104 x0x0_11 + -0.0048 x0x0x0_11 + 0.0010 x0x0_111 + 0.0046 x0x0x0_111 + -0.0042 x0x0_1111 + -0.0021 x0x0x0_1111


### Dataset 2

In [13]:
time_subsample = 10
full_dat = np.zeros((5000,6000))
fileStr = "2021-11-01T2321_unfiltered"


k = 0
for i in np.arange(21,26):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    


In [14]:
working_dat = full_dat/np.std(full_dat)#*10**6
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [15]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6695 x0_1 + 0.3843 x0_111


In [16]:
thresh = 1.e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6617 x0_1 + 0.3828 x0_111


In [17]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6690 x0_1 + -0.0099 x0_11 + 0.3735 x0_111 + -0.0033 x0x0_11 + -0.0088 x0x0_111 + 0.0085 x0x0x0_111


In [18]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6562 x0_1 + -0.0108 x0_11 + 0.3785 x0_111 + -0.0017 x0_1111 + -0.0058 x0x0_1 + 0.0128 x0x0x0_1 + -0.0074 x0x0_11 + 0.0055 x0x0x0_11 + -0.0177 x0x0_111 + 0.0088 x0x0x0_111 + 0.0015 x0x0x0_1111


In [19]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = -0.0010 1 + 0.0041 x0 + -0.0046 x0x0 + 0.6563 x0_1 + -0.0117 x0_11 + 0.3768 x0_111 + -0.0052 x0_1111 + -0.0060 x0x0_1 + 0.0068 x0x0x0_1 + -0.0071 x0x0_11 + 0.0068 x0x0x0_11 + -0.0112 x0x0_111 + 0.0088 x0x0x0_111 + -0.0020 x0x0_1111 + 0.0023 x0x0x0_1111


## Drift Removal Load

### Dataset 1

In [7]:
fileStr = "2021-11-01T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [9]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6939 x0_1 + 0.3295 x0_111


In [14]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6836 x0_1 + 0.3212 x0_111


In [10]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.7132 x0_1 + 0.3440 x0_111 + -0.0163 x0x0x0_1 + -0.0051 x0x0x0_111


In [11]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.7093 x0_1 + 0.0083 x0_11 + 0.3323 x0_111 + -0.0102 x0x0x0_1 + -0.0096 x0x0x0_111


In [13]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.7027 x0_1 + 0.3333 x0_111 + 0.0104 x0x0_1 + -0.0084 x0x0x0_1 + -0.0059 x0x0x0_111 + 0.0018 x0x0_1111


### Dataset 2

In [15]:
fileStr = "2021-11-01T2321_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [16]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6659 x0_1 + 0.3176 x0_111


In [17]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6857 x0_1 + 0.3270 x0_111


In [18]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6808 x0_1 + 0.3222 x0_111


In [19]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6768 x0_1 + 0.3100 x0_111 + 0.0092 x0x0_1 + 0.0069 x0x0_111


In [20]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6867 x0_1 + 0.0102 x0_11 + 0.3169 x0_111 + 0.0033 x0_1111 + -0.0142 x0x0_1 + -0.0042 x0x0_11 + -0.0166 x0x0x0_11 + -0.0041 x0x0_111 + -0.0021 x0x0_1111 + -0.0045 x0x0x0_1111


### Dataset 3

In [21]:
fileStr = "2021-11-02T0511_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [22]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6758 x0_1 + 0.3260 x0_111


In [23]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6973 x0_1 + 0.3310 x0_111


In [24]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.7002 x0_1 + 0.3272 x0_111


In [25]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6879 x0_1 + 0.3161 x0_111 + 0.0133 x0x0_1 + 0.0029 x0x0_111


In [26]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6847 x0_1 + -0.0027 x0_11 + 0.3173 x0_111 + -0.0024 x0_1111 + -0.0052 x0x0_1 + -0.0055 x0x0x0_1 + -0.0050 x0x0_11 + 0.0023 x0x0x0_11 + -0.0059 x0x0_111 + -0.0017 x0x0x0_111


### Dataset 4

In [27]:
fileStr = "2021-11-02T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [28]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6140 x0_1 + 0.2907 x0_111


In [29]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6234 x0_1 + 0.2926 x0_111 + 0.0211 x0x0_1


In [30]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6369 x0_1 + 0.3017 x0_111 + -0.0161 x0x0x0_1 + -0.0070 x0x0x0_111


In [31]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6159 x0_1 + 0.2876 x0_111 + 0.0093 x0x0_11


In [32]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6127 x0_1 + -0.0103 x0_11 + 0.2838 x0_111 + 0.0147 x0x0_1 + -0.0024 x0x0x0_1 + 0.0018 x0x0_11 + 0.0081 x0x0x0_11 + 0.0043 x0x0_111 + 0.0026 x0x0x0_1111


## Multifilt

### Dataset 1

In [20]:
fileStr = "2021-11-01T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [34]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9959 x0_1


In [35]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9969 x0_1 + 0.0869 x0_111 + 0.0632 x0x0x0_111


In [36]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9971 x0_1 + 0.0827 x0_111 + 0.0636 x0x0x0_111


In [37]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9971 x0_1 + 0.0851 x0_111 + 0.0638 x0x0x0_111


In [38]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9954 x0_1 + 0.0843 x0_111 + 0.0017 x0x0x0_1 + 0.0015 x0x0_111 + 0.0686 x0x0x0_111


In [21]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9953 x0_1 + 0.0858 x0_111 + 0.0017 x0x0x0_1 + -0.0015 x0x0_111 + 0.0719 x0x0x0_111


### Dataset 2

In [22]:
fileStr = "2021-11-01T2321_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [40]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9964 x0_1


In [41]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9971 x0_1 + 0.0992 x0_111 + 0.0621 x0x0x0_111


In [42]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9973 x0_1 + 0.0972 x0_111 + 0.0626 x0x0x0_111


In [43]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9974 x0_1 + 0.0976 x0_111 + 0.0640 x0x0x0_111


In [44]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9955 x0_1 + 0.0975 x0_111 + 0.0016 x0x0x0_1 + 0.0654 x0x0x0_111


In [45]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9956 x0_1 + -0.0012 x0_11 + 0.0972 x0_111 + 0.0016 x0x0x0_1 + -0.0018 x0x0_111 + 0.0700 x0x0x0_111


### Dataset 3

In [23]:
fileStr = "2021-11-02T0511_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [47]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9966 x0_1


In [48]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9968 x0_1 + 0.1074 x0_111 + 0.0525 x0x0x0_111


In [49]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9976 x0_1 + 0.1035 x0_111 + 0.0557 x0x0x0_111


In [50]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9971 x0_1 + 0.1022 x0_111 + 0.0568 x0x0x0_111


In [51]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9959 x0_1 + 0.1000 x0_111 + 0.0013 x0x0x0_1 + 0.0630 x0x0x0_111


In [24]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9959 x0_1 + -0.0023 x0_11 + 0.1032 x0_111 + 0.0013 x0x0x0_1 + 0.0018 x0x0_111 + 0.0637 x0x0x0_111


### Dataset 4

In [25]:
fileStr = "2021-11-02T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [53]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9978 x0_1 + 0.2165 x0_111


In [54]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9982 x0_1 + 0.1826 x0_111 + 0.0607 x0x0x0_111


In [55]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9984 x0_1 + 0.1815 x0_111 + 0.0616 x0x0x0_111


In [56]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9983 x0_1 + 0.1799 x0_111 + 0.0612 x0x0x0_111


In [57]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9969 x0_1 + -0.0024 x0_11 + 0.1774 x0_111 + 0.0016 x0x0x0_1 + 0.0709 x0x0x0_111 + -0.0019 x0x0x0_1111


In [26]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9963 x0_1 + -0.0013 x0_11 + 0.1737 x0_111 + 0.0011 x0_1111 + 0.0016 x0x0x0_1 + -0.0040 x0x0_111 + 0.0728 x0x0x0_111 + -0.0011 x0x0_1111


## Multifilt (radius 10 smooth)

### Dataset 1

In [58]:
fileStr = "2021-11-01T2311_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [59]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9975 x0_1


In [60]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9976 x0_1


In [61]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9962 x0_1 + -0.0011 x0_111 + 0.0016 x0x0x0_1


In [62]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9960 x0_1 + -0.0012 x0_111 + 0.0016 x0x0x0_1


In [63]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9959 x0_1 + -0.0011 x0_111 + 0.0017 x0x0x0_1


### Dataset 2

In [64]:
fileStr = "2021-11-01T2321_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [65]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9978 x0_1


In [66]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9977 x0_1


In [67]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9965 x0_1 + -0.0014 x0_111 + 0.0013 x0x0x0_1


In [68]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9966 x0_1 + -0.0014 x0_111 + 0.0012 x0x0x0_1


In [69]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9965 x0_1 + -0.0014 x0_111 + 0.0012 x0x0x0_1


### Dataset 3

In [70]:
fileStr = "2021-11-02T0511_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [71]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9979 x0_1


In [72]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9979 x0_1


In [73]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9977 x0_1 + -0.0014 x0_111


In [74]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9970 x0_1 + -0.0013 x0_111


In [75]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9968 x0_1 + -0.0013 x0_111


### Dataset 4

In [76]:
fileStr = "2021-11-02T2311_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [77]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9985 x0_1


In [78]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9985 x0_1


In [79]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9985 x0_1 + -0.0021 x0_111


In [80]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9978 x0_1 + -0.0020 x0_111


In [81]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9978 x0_1 + -0.0020 x0_111
